# **Materia**: Aprendizaje Automático
## **Año**: 2025 - 1º Cuatrimestre
### **Alumno**: SANCHEZ Sergio Andrés

# Presentación de Objetivo del Proyecto

Desarrollar un modelo de Aprendizaje Automático que permita estimar el tiempo de vida restante de explotación de un yacimiento de turba en Tierra del Fuego, promoviendo una gestión sostenible del recurso.

El problema que se busca resolver es de regresión, ya que la variable objetivo será una estimación continua del tiempo (en meses, años o décadas) restante hasta que se agote el yacimiento, en función del volumen total estimado de turba y del ritmo de extracción y comercialización histórico.


# Importancia y Relevancia del problema abordado

La extracción de turba en la provincia de Tierra del Fuego representa una actividad productiva clave, especialmente en zonas rurales. La turba es un recurso natural no renovable en escalas humanas de tiempo y su extracción está condicionada por factores climáticos, estacionales y económicos.

Con una estacionalidad marcada (más actividad en verano que en invierno), la explotación turbera requiere una planificación precisa para evitar la sobreexplotación y promover la sostenibilidad del recurso. Actualmente, las decisiones sobre tiempos de explotación se basan en cálculos aproximados o estimaciones estáticas. Un modelo predictivo que estime cuánto tiempo resta de explotación puede apoyar políticas públicas, decisiones empresariales y controles ambientales.


# Descripción del Dataset

**Origen:** El Dataset proviene de una base de datos en Postgre de la Dirección General de Desarrollo Minero dependiente del Ministerio de Producción y Ambiente del Gobierno de la Provincia de Tierra del Fuego, Antártida e Islas del Atlántico Sur.

**Fecha de disponibilidad del Dataset:** Viernes 06 de junio de 2025 – Segunda Versión Dataset.

El Dataset provisto contempla expedientes tramitados por explotación de Turba Rubia y Negra por cada uno de los yacimientos desde enero de 2020 a diciembre de 2024.

Este Dataset cuenta con 1635 filas y 27 columnas con las siguientes características:

| Variable               | Tipo        | Definición                                                                 |
|------------------------|-------------|---------------------------------------------------------------------------|
| Area                  | Numérica    | Superficie del yacimiento, en hectáreas                                   |
| Vol_N                 | Numérica    | Volumen de turba negra a 2023                                             |
| Vol_R                 | Numérica    | Volumen de turba rubia a 2023                                             |
| Vol_total             | Numérica    | Volumen total de turba a 2023                                             |
| Fecha                 | Numérica    | Año de cuantificación (2023)                                              |
| id                    | Numérica    | valor de identificación (no tener en cuenta)                              |
| _uid_                 | Numérica    | valor de identificación (no tener en cuenta)                              |
| poligono              | Numérica    | número del polígono                                                       |
| id_2                  | Numérica    | valor de identificación (no tener en cuenta)                              |
| producto              | Categórica  | tipo de mineral extraído                                                  |
| num_expte             | Numérica    | número de expediente de regalías mineras                                  |
| ano                   | Numérica    | año de declaración jurada                                                 |
| mes                   | Numérica    | mes de declaración jurada                                                 |
| volumen_produccion    | Numérica    | volumen de mineral extraído                                               |
| trimestre             | Numérica    | trimestre de la declaración jurada                                        |
| regalias              | Numérica    | importe de regalías mineras liquidado                                     |
| tasa_insp_fisc        | Numérica    | importe de tasas de fiscalización liquidado                               |
| ano_comer             | Numérica    | año de comercialización territorio nacional continental                    |
| mes_comer             | Numérica    | mes de comercialización a territorio nacional continental                  |
| productor_comer       | Numérica    | número de productor                                                       |
| producto_comer        | Numérica    | producto comercializado                                                   |
| volumen_comercializado| Numérica    | volumen comercializado                                                    |
| valor_fob_usd         | Numérica    | valor FOB expresado en dólares (valor de mineral puesto en aduana)        |
| valor_fob_ars         | Numérica    | valor FOB expresado en pesos (valor de mineral puesto en aduana)          |
| tasas_comercial       | Numérica    | tasas abonadas por emisión de certificado de exportación                  |
| tasa_cambio           | Numérica    | relación peso/dólar al emitir el certificado de exportación               |
| sin_comercializacion  | Categórica  | valor TRUE (no comercializó) o FALSE (comercializó)

De este Dataset anonimizado, se estimará el tiempo de vida de cada uno de los yacimientos en base a saber su volumen total actual y las extracciones realizadas en los periodos que van desde Enero 2020 a Diciembre 2024 (5 años exactos).

Para el análisis a realizar, tendremos en cuenta solo aquellos yacimientos que han tenido producción en el periodo mencionado, es decir, que hayan tenido o estén en actividad informada.


# Desarrollo del Modelo


Para este modelo se utilizó:

* Algoritmo: Random Forest Regressor
* Tipo de Modelo: Aprendizaje supervisado, Regresión
* Variable Objetivo: meses_restantes_estimados (vida útil en meses)
* Variable predictora principal: extraccion_prom_mensual (media móvil de extracción de los últimos 6 meses).
* Transformación aplicada: Log-transformación de la variable objetivo para estabilizar la varianza y mejorar la distribución.

# Hiperparámetros utilizados

* n_estimators = 100: número de árboles en el bosque.
* random_state = 42: para reproducibilidad.

Se utilizó el valor por defecto de max_depth para permitir que cada árbol crezca libremente, maximizando el ajuste local.


# Métricas utilizadas para evaluar el modelo

* **MAE** (Error Absoluto Medio)
* **RMSE** (Raíz del Error Cuadrático Medio)
* **R2** (Coeficiente de Determinación)

# Importación de Librerías

In [ ]:
# --- IMPORTACIÓN DE LIBRERÍAS ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carga de Datos

In [ ]:
# --- CARGA DE DATOS ---

from google.colab import drive
drive.mount('/content/drive')

ruta = "/content/drive/My Drive/CPSMA/2025/Parcial/dataset.csv"

df = pd.read_csv(ruta, sep=";")

# Guardo una copia del DataFrame original

In [ ]:
# Guardo una copia del DataFrame original
df_original = df.copy()

df

# Limpieza de Datos

In [ ]:
# Elimino las columnas que no voy a utilizar

df = df.drop(columns=["Fecha", "id", "_uid_", "id_2", "producto", "num_expte", "trimestre", "regalias",
                      "tasa_insp_fisc", "ano_comer", "mes_comer", "productor_comer",
                      "producto_comer", "volumen_comercializado", "valor_fob_usd", "valor_fob_ars",
                      "tasas_comercial", "tasa_cambio", "sin_comercializacion"])


df.info()

df

In [ ]:
# ------------ Limpio filas por poligono vacio ------------

# Filtro las filas que serán eliminadas (donde 'poligono' está vacío o NaN)
filas_a_eliminar = df[df["poligono"].isna()]

# Reemplazo strings vacíos por NaN
df["poligono"] = df["poligono"].replace('', np.nan)

# Muestro las filas que serán eliminadas
print("Filas que serán eliminadas (por 'poligono' en blanco o NaN):")
display(filas_a_eliminar)

# Elimino las filas del DataFrame original
df = df.dropna(subset=["poligono"])

# Muestro la cantidad de filas eliminadas
print(f"\nTotal de filas eliminadas: {len(filas_a_eliminar)}")

# Confirmo cuántas filas quedan
print(f"\nFilas restantes luego de limpieza: {len(df)}")

print(df.dtypes)

In [ ]:
# Paso los object a FLOAT

# Me aseguro de tener los tipos de datos que corresponde en cada característica

# Hago una copia por seguridad antes de trabajar y evitar Warnings
df = df.copy()

# Ver todas las columnas de tipo object
df.select_dtypes(include='object').columns

cols_a_convertir = ["Area", "Vol_N", "Vol_R", "Vol_total", "volumen_produccion"]
for col in cols_a_convertir:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Verificación final de los tipos de datos
print(df.dtypes)

In [ ]:
# Convierto columnas "poligono", "ano", "mes" a enteros
columnas_enteras = ["poligono", "ano", "mes"]
for col in columnas_enteras:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0).astype(int)

# Verificación final de los tipos de datos
print(df.dtypes)

In [ ]:
# Creo columnas de fecha
df["fecha"] = pd.to_datetime(
    df["ano"].astype(str) + "-" + df["mes"].astype(str).str.zfill(2),
    format="%Y-%m",
    errors="coerce"
)
df["ano_mes"] = df["fecha"].dt.to_period("M")

# Verificación final de los tipos de datos
print(df.dtypes)

df

In [ ]:
# ------------ Limpio filas por volumen_produccion = 0  ------------

# Filtro las filas que serán eliminadas (donde 'volumen_produccion' = 0)

# Cantidad de Filas Antes de la limpieza
filas_antes = df.shape[0]

# Filtro y dejo solo las que volumen_produccion = 0
df = df[df["volumen_produccion"] != 0.0]

# Cantidad de Filas después de la limpieza
filas_despues = df.shape[0]

print(f"Filas eliminadas: {filas_antes - filas_despues}")

df

In [ ]:
# Detección de outliers en volumen_produccion con IQR y decidir si eliminarlos o transformarlos

Q1 = df["volumen_produccion"].quantile(0.25)
Q3 = df["volumen_produccion"].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
df_cleaned = df[(df["volumen_produccion"] >= limite_inferior) & (df["volumen_produccion"] <= limite_superior)]

In [ ]:
# Muestro valor de cada dato obtenido

print("Q1 (25%):", Q1)
print("Q3 (75%):", Q3)
print("IQR:", IQR)
print("Límite inferior:", limite_inferior)
print("Límite superior:", limite_superior)

In [ ]:
# Visualizo los outliers (Antes de eliminar)


sns.boxplot(data=df, y="volumen_produccion")
plt.title("Boxplot de volumen_produccion (Antes de eliminar Outliers)")
plt.show()

In [ ]:
# Elimino Outliers

outliers = df_cleaned = df[(df["volumen_produccion"] >= limite_inferior) & (df["volumen_produccion"] <= limite_superior)]

df

In [ ]:
# Visualizo outliers (Después de eliminar)

sns.boxplot(data=df_cleaned, y="volumen_produccion")
plt.title("Boxplot de Ingresos (Después de eliminar Outliers)")
plt.show()

In [ ]:
# Copio el Dataframe de limpieza al Dataframe df
df = df_cleaned.copy()

In [ ]:
# --- AGRUPACIÓN POR POLÍGONO Y MES ---
df_prod = df.groupby(["poligono", "ano_mes"]).agg({
    "volumen_produccion": "sum",
    "Area": "first",
    "Vol_N": "first",
    "Vol_R": "first",
    "Vol_total": "first"
}).reset_index()

# Cálculo de extracción acumulada
df_prod["volumen_acumulado"] = df_prod.groupby("poligono")["volumen_produccion"].cumsum()

# Cálculo de volumen restante
df_prod["volumen_restante"] = df_prod["Vol_total"] - df_prod["volumen_acumulado"]

# Promedio móvil de extracción
df_prod["extraccion_prom_mensual"] = df_prod.groupby("poligono")["volumen_produccion"].transform(lambda x: x.rolling(window=6, min_periods=1).mean())

# Reemplazo 0 por NaN antes del cálculo
df_prod.loc[df_prod["extraccion_prom_mensual"] == 0, "extraccion_prom_mensual"] = np.nan

# Estimo meses restantes (evita división por cero)
df_prod["meses_restantes_estimados"] = df_prod["volumen_restante"] / df_prod["extraccion_prom_mensual"]
df_prod["meses_restantes_estimados"] = df_prod["meses_restantes_estimados"].clip(lower=0)

# Elimino las filas del DataFrame produccion
df_prod = df_prod.dropna(subset=["extraccion_prom_mensual"])

df_prod

In [ ]:
# Reemplazo 0 en extraccion_prom_mensual por NaN
df_prod.loc[df_prod["extraccion_prom_mensual"] == 0, "extraccion_prom_mensual"] = np.nan

# Construyo el Modelo - RandomForestRegressor

In [ ]:
# --- CONSTRUCCIÓN DEL MODELO ---
df_modelo = df_prod.dropna(subset=["meses_restantes_estimados", "extraccion_prom_mensual"])
X = df_modelo[["Area", "Vol_N", "Vol_R", "Vol_total", "extraccion_prom_mensual"]]
y = df_modelo["meses_restantes_estimados"]

print(f"Filas en X: {len(X)}")
print(f"Filas en y: {len(y)}")
print("¿Hay valores nulos en X?")
print(X.isnull().sum())

print("Valores nulos en 'extraccion_prom_mensual':", df_prod["extraccion_prom_mensual"].isna().sum())

print("Valores nulos en 'meses_restantes_estimados':", df_prod["meses_restantes_estimados"].isna().sum())

print("Tamaño de df_prod:", df_prod.shape)
print("Tamaño de df_modelo antes de dropna:", df_prod[["meses_restantes_estimados", "extraccion_prom_mensual"]].shape)

# División entrenamiento / prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Confirmar que X e y no tengan NaN
print("¿Hay NaN en X?", X.isna().sum())
print("¿Hay NaN en y?", y.isna().sum())

# Entrenar modelo
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Predicción
y_pred = modelo.predict(X_test)

# Evaluación
print("MAE:", mean_absolute_error(y_test, y_pred))

#rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

print("R² Score:", r2_score(y_test, y_pred))

In [ ]:
# Importancia de atributos
importancias = pd.Series(modelo.feature_importances_, index=X.columns)
importancias.sort_values(ascending=True).plot(kind="barh")
plt.title("Importancia de Atributos")
plt.show()

# Guardar resultados
df_modelo["meses_estimados_modelo"] = modelo.predict(X)
df_modelo.to_csv("predicciones_vida_util.csv", index=False)

In [ ]:
# Analizar errores del modelo - Calidad de las predicciones
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Real vs. Predicho")
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')  # línea ideal
plt.show()

In [ ]:
# Analizo los polígonos extremos
df_modelo["error_meses"] = abs(df_modelo["meses_restantes_estimados"] - df_modelo["meses_estimados_modelo"])
df_modelo.sort_values("error_meses", ascending=False).head(10)

In [ ]:
# Polígonos con más registros
poligonos_mas_datos = df_prod["poligono"].value_counts().head(5).index.tolist()
print("Polígonos con más datos:", poligonos_mas_datos)

In [ ]:
# Visualizo el impacto
df_modelo[["meses_restantes_estimados", "meses_estimados_modelo", "error_meses"]].describe()

In [ ]:
# Tabla resumén por polígono

# Tomar el último registro de cada polígono
vida_util_por_poligono = df_modelo.sort_values("ano_mes").groupby("poligono").tail(1)

# Seleccionar columnas relevantes
vida_util_por_poligono = vida_util_por_poligono[["poligono", "ano_mes", "volumen_restante", "extraccion_prom_mensual", "meses_restantes_estimados", "meses_estimados_modelo", "error_meses"]]

# Ordenar de mayor a menor estimación
vida_util_por_poligono = vida_util_por_poligono.sort_values("poligono", ascending=True)

# Mostrar
vida_util_por_poligono

In [ ]:
# Gráfico de Barras Vertical (Ordenado por poligono)

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
vida_util_por_poligono_plot = vida_util_por_poligono.sort_values("poligono", ascending=True)

plt.bar(vida_util_por_poligono_plot["poligono"].astype(str),
        vida_util_por_poligono_plot["meses_estimados_modelo"],
        color='skyblue')

plt.ylabel("Meses estimados restantes")
plt.xlabel("Polígono")
plt.title("Estimación de vida útil de yacimientos (por polígono)")
plt.xticks(rotation=90)  # Rotar etiquetas si son muchas
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de Barras Horizontal (Ordenado por meses_estimados_modelo)

plt.figure(figsize=(14, 6))
vida_util_por_poligono_plot = vida_util_por_poligono.sort_values("meses_estimados_modelo", ascending=True)

plt.bar(vida_util_por_poligono_plot["poligono"].astype(str),
         vida_util_por_poligono_plot["meses_estimados_modelo"],
         color='skyblue')

plt.ylabel("Meses estimados restantes")
plt.xlabel("Polígono")
plt.title("Estimación de vida útil de yacimientos (por polígono)")
plt.xticks(rotation=90)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
# Graficar para un polígono

import matplotlib.dates as mdates

def graficar_poligono(df, poligono_id):
    df_pol = df[df["poligono"] == poligono_id].sort_values("ano_mes")
    df_pol["fecha_dt"] = df_pol["ano_mes"].dt.to_timestamp()

    fig, ax1 = plt.subplots(figsize=(12,6))

    # Eje 1: Volumen de producción
    ax1.plot(df_pol["fecha_dt"], df_pol["volumen_produccion"], label="Volumen de Producción", color="tab:blue")
    ax1.plot(df_pol["fecha_dt"], df_pol["extraccion_prom_mensual"], label="Extracción Promedio (6M)", color="tab:cyan", linestyle="--")
    ax1.set_ylabel("Volumen (m³)", color="tab:blue")
    ax1.tick_params(axis='y', labelcolor="tab:blue")

    # Formato de fechas
    ax1.xaxis.set_major_locator(mdates.YearLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.xticks(rotation=45)

    # Eje 2: Meses restantes
    ax2 = ax1.twinx()
    ax2.plot(df_pol["fecha_dt"], df_pol["meses_restantes_estimados"], label="Meses Restantes Estimados", color="tab:red")
    ax2.set_ylabel("Meses restantes", color="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:red")

    # Título y leyendas
    plt.title(f"Evolución de Extracción y Vida Útil Estimada - Polígono {poligono_id}")
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Elegí uno de los polígonos con más datos
graficar_poligono(df_prod, poligonos_mas_datos[0])

In [ ]:
# Estadísticas de la variable objetivo
print(df_modelo["meses_restantes_estimados"].describe())

# Histograma para ver la distribución
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.hist(df_modelo["meses_restantes_estimados"], bins=30, edgecolor='k')
plt.title("Distribución de Meses Restantes Estimados")
plt.xlabel("Meses restantes")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
# Box-Plot Horizontal de la característica "meses_restantes_estimados"

plt.figure(figsize=(12,8))
plt.boxplot(df_modelo["meses_restantes_estimados"], vert=False)
plt.title("Boxplot de Meses Restantes Estimados")
plt.xlabel("Meses restantes")
plt.show()

In [ ]:
# Saco medidas para observar los outliers y elimanr los innecesarios
Q1 = df_modelo["meses_restantes_estimados"].quantile(0.25)
Q3 = df_modelo["meses_restantes_estimados"].quantile(0.75)
IQR = Q3 - Q1

limite_inf = Q1 - 1.5 * IQR
limite_sup = Q3 + 1.5 * IQR

outliers = df_modelo[
    (df_modelo["meses_restantes_estimados"] < limite_inf) |
    (df_modelo["meses_restantes_estimados"] > limite_sup)
]
print(f"Outliers detectados: {len(outliers)} filas")
display(outliers.head())

In [ ]:
# Filtro las filas que NO son outliers
df_modelo_sin_outliers = df_modelo[
    (df_modelo["meses_restantes_estimados"] >= limite_inf) &
    (df_modelo["meses_restantes_estimados"] <= limite_sup)
]

# Confirmo la cantidad de filas después de la limpieza
print(f"Filas sin outliers: {len(df_modelo_sin_outliers)}")

# Reemplazo el DataFrame original y lo dejo sin los outliers
df_modelo = df_modelo_sin_outliers.copy()

# Genero el Modelo por Transformación Logarítmica

In [ ]:
import numpy as np

# Transformación logarítmica (añado 1 para evitar log(0))
df_modelo["meses_log"] = np.log1p(df_modelo["meses_restantes_estimados"])

# Volver a entrenar con y = meses_log en lugar de meses_restantes_estimados
X = df_modelo[["extraccion_prom_mensual"]]
y_log = df_modelo["meses_log"]

modelo_log = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_log.fit(X, y_log)

# Predecir y volver a transformar la predicción
pred_log = modelo_log.predict(X)
pred_unlog = np.expm1(pred_log)

# Calcular MAE en la escala original
from sklearn.metrics import mean_absolute_error
mae_unlog = mean_absolute_error(
    df_modelo["meses_restantes_estimados"],
    pred_unlog
)
print("MAE después de log-transform & back-transform:", mae_unlog)

from sklearn.metrics import mean_squared_error

# Calcular RMSE en la escala original
rmse_unlog = np.sqrt(mean_squared_error(
    df_modelo["meses_restantes_estimados"],
    pred_unlog
))
print("RMSE después de log-transform & back-transform:", rmse_unlog)

# Después de calcular pred_unlog y asignarlo a df_modelo["meses_estimados_log_modelo"]

from sklearn.metrics import r2_score

r2_original = r2_score(df_modelo["meses_restantes_estimados"], df_modelo["meses_estimados_modelo"])
df_modelo["meses_estimados_log_modelo"] = pred_unlog
r2_log = r2_score(df_modelo["meses_restantes_estimados"], df_modelo["meses_estimados_log_modelo"])

print(f"R² para predicción log-transformada y revertida: {r2_log:.4f}")

Si bien parecen mejorar tanto en MAE como en RMSE, el R2 empeoró muchísimo.

In [ ]:
# Tabla resumén por polígono

# Crear columna de predicción revertida
df_modelo["meses_estimados_log_modelo"] = pred_unlog

# Calcular el error absoluto
df_modelo["error_log"] = abs(df_modelo["meses_restantes_estimados"] - df_modelo["meses_estimados_log_modelo"])

# Seleccionar columnas clave para el resumen
resumen_log = df_modelo[[
    "poligono", "volumen_restante", "extraccion_prom_mensual",
    "meses_restantes_estimados", "meses_estimados_log_modelo", "error_log"
]]

# Ordenar por número de polígono en orden ascendente
resumen_log = resumen_log.sort_values(by="error_log", ascending=True)

# Mostrar la tabla
display(resumen_log)

In [ ]:
# Tomar el último registro de cada polígono
vida_util_log_por_poligono = df_modelo.sort_values("ano_mes").groupby("poligono").tail(1)

# Seleccionar columnas relevantes
vida_util_log_por_poligono = vida_util_log_por_poligono[[
    "poligono", "ano_mes", "volumen_restante", "extraccion_prom_mensual",
    "meses_restantes_estimados", "meses_estimados_log_modelo", "error_log"
]]

# Ordenar de mayor a menor por número de polígono
vida_util_log_por_poligono = vida_util_log_por_poligono.sort_values("poligono", ascending=True)

# Mostrar tabla
display(vida_util_log_por_poligono)

Aparte de haber empeorado muchísimo el R², también tengo 2 polígonos menos (3 y 18).

In [ ]:
import pandas as pd

# Crear una nueva columna con rangos de meses
df_modelo["rango_meses"] = pd.cut(
    df_modelo["meses_restantes_estimados"],
    bins=[0, 100, 500, 1000, 5000, 10000, np.inf],
    labels=["0-100", "100-500", "500-1000", "1000-5000", "5000-10000", "10000+"]
)

# Calcular errores de ambos modelos
df_modelo["error_original"] = abs(df_modelo["meses_restantes_estimados"] - df_modelo["meses_estimados_modelo"])
df_modelo["error_log"] = abs(df_modelo["meses_restantes_estimados"] - df_modelo["meses_estimados_log_modelo"])

# Agrupar por rango y calcular estadísticas
comparacion = df_modelo.groupby("rango_meses", observed=False).agg({
    "error_original": ["mean", "median", "count"],
    "error_log": ["mean", "median"]
}).reset_index()

# Renombrar columnas para claridad
comparacion.columns = [
    "Rango de meses",
    "MAE original (media)",
    "Mediana error original",
    "Cantidad",
    "MAE log-transform",
    "Mediana error log"
]

# Mostrar la tabla comparativa
display(comparacion)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definimos los rangos de meses restantes
bins = [0, 100, 500, 1000, 5000, 10000, np.inf]
labels = ["0-100", "100-500", "500-1000", "1000-5000", "5000-10000", "10000+"]

# Clasificamos cada fila según su rango de meses restantes
df_modelo["rango_meses"] = pd.cut(df_modelo["meses_restantes_estimados"], bins=bins, labels=labels, right=False)

# Calculamos MAE por rango para modelo original
mae_original = df_modelo.groupby("rango_meses", observed=False)["error_meses"].mean()

# Calculamos MAE por rango para modelo log-transform
mae_log = df_modelo.groupby("rango_meses", observed=False)["error_log"].mean()

x = np.arange(len(labels))
width = 0.35

plt.figure(figsize=(12, 6))

bars1 = plt.bar(x - width/2, mae_original, width, label='MAE Original', color='steelblue')
bars2 = plt.bar(x + width/2, mae_log, width, label='MAE Log-Transform', color='darkorange')

plt.ylabel('MAE')
plt.xlabel('Rango de Meses')
plt.title('Comparación de MAE por Rango de Meses Estimados')
plt.xticks(x, labels)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Mostrar valores numéricos encima de cada barra
for i in range(len(x)):
    plt.text(x[i] - width/2, mae_original.iloc[i] + 30, f"{mae_original.iloc[i]:.0f}", ha='center', fontsize=9)
    plt.text(x[i] + width/2, mae_log.iloc[i] + 30, f"{mae_log.iloc[i]:.0f}", ha='center', fontsize=9)

plt.tight_layout()
plt.show()

En este gráfico se puede observar para concluir que:

*	Para los rangos bajos (0-100, 100-500, etc.), el modelo original tiene errores mucho menores que el modelo log-transformado.
*	A medida que aumenta el rango, especialmente desde los 1000 meses en adelante, el modelo log-transformado comienza a producir errores mucho mayores.

l Modelo de Transformación Logarítmica  puede suavizar la influencia de valores extremos en el entrenamiento, pero al revertirla puede amplificar los errores en las predicciones grandes. Esto sugiere que el modelo original está mejor calibrado para todo el rango, especialmente para valores grandes, mientras que el log-transformado puede no estar capturando correctamente la magnitud de las predicciones en rangos altos.


# Conclusión Final

El modelo de Random Forest Regressor sin transformación logarítmica fue el que presentó el mejor rendimiento global, con un R² de 0.96, lo que indica una excelente capacidad explicativa de la variabilidad de los datos. Además, mostró un error absoluto medio (MAE) de aproximadamente 837 meses y un RMSE de 3840, valores coherentes con la escala y dispersión de la variable objetivo.

Por otro lado, el modelo con transformación logarítmica logró un MAE más bajo (711) y una notable reducción del RMSE (1432), pero a costa de una fuerte caída en el R² (0.57). Esta baja en el poder explicativo, sumada al análisis por rangos de meses restantes, donde el modelo original superó consistentemente al transformado, refuerza la decisión de mantener el modelo sin logaritmo como el más confiable.

Sin embargo, es importante remarcar que el dataset cuenta con solo cinco años de información histórica (Enero 2020 a Diciembre 2024), lo cual resulta escaso frente a estimaciones que se expresan en escalas de cientos o incluso miles de meses. Esta limitación temporal reduce la capacidad del modelo para aprender patrones a largo plazo y afecta su fiabilidad cuando proyecta escenarios muy alejados en el tiempo.

A pesar de la mejora en algunos indicadores puntuales, la transformación logarítmica no aportó beneficios generalizados al rendimiento del modelo. El modelo original de Random Forest, sin transformación, es el más robusto y balanceado para estimar la vida útil restante de los yacimientos de turba. No obstante, debe considerarse que la brevedad del histórico disponible condiciona la precisión de las estimaciones, especialmente en horizontes temporales largos.
